In [ ]:
import os
import sys

spark_path = os.environ['SPARK_HOME']
sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.10.9-src.zip")

import findspark
findspark.init()

import pyspark

In [ ]:
number_cores = 8
memory_gb = 8
conf = (pyspark.SparkConf().setMaster('local[{}]'.format(number_cores)).set('spark.driver.memory', '{}g'.format(memory_gb)))
sc = pyspark.SparkContext(conf=conf)

In [ ]:
from pyspark.ml.linalg import Vectors
data = [(Vectors.dense([0.0, 0.0]), 2.0), (Vectors.dense([1.0, 1.0]), 2.0),
        (Vectors.dense([9.0, 8.0]), 2.0), (Vectors.dense([8.0, 9.0]), 2.0)]

In [ ]:
from pyspark.sql import SQLContext
sqlContext= SQLContext(sc)
df = sqlContext.createDataFrame(data, ["features", "weighCol"])

In [ ]:
from pyspark.ml.clustering import KMeans
kmeans = KMeans(k=2)
kmeans.setSeed(1)
kmeans.setWeightCol("weighCol")
kmeans.setMaxIter(10)
kmeans.getMaxIter()
10

In [ ]:
kmeans.clear(kmeans.maxIter)
model = kmeans.fit(df)
model.getDistanceMeasure()

In [ ]:
model.setPredictionCol("newPrediction")
model.predict(df.head().features)

In [ ]:
centers = model.clusterCenters()
len(centers)

In [ ]:
transformed = model.transform(df).select("features", "newPrediction")
rows = transformed.collect()
rows

## Run the model for Bisecting KMeans based on the API

## Reddit Crypto Data

- Download the [Reddit Crypto data](https://www.cs.wcupa.edu/lngo/data2/reddit_crypto.zip)
- Unzip the crypto data.
- Review [the metadata](https://www.kaggle.com/pavellexyr/reddit-cryptocurrency-data-for-august-2021)
- Perform a K-mean clustering on the text of posts and comments. Address the followings:
  - What data point to remove, what not to remove?
  - How to clean up text data?
  - How to make the resulting cluster data more meaningful?